# 检索机器人

## Step1 读取faq数据

In [1]:
import pandas as pd

data = pd.read_csv("./law_faq.csv")
data.head()

,title,reply
0,在法律中定金与订金的区别订金和定金哪个受,“定金”是指当事人约定由一方向对方给付的，作为债权担保的一定数额的货币，它属于一种法律上的担...
1,盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体,盗窃罪的客体要件本罪侵犯的客体是公私财物的所有权。侵犯的对象，是国家、集体或个人的财物，一般...
2,非法微整形机构构成非法经营罪吗,符合要件就有可能。非法经营罪，是指未经许可经营专营、专卖物品或其他限制买卖的物品，买卖进出口...
3,入室持刀行凶伤人能不能判刑,对于入室持刀伤人涉嫌故意伤害刑事犯罪，一经定罪，故意伤害他人身体的，处三年以下有期徒刑、拘役...
4,对交通事故责任认定书不服怎么办，交通事故损,事故认定书下发后，如果你对认定不满意，可在接到认定书3日内到上一级公安机关复议。


## Step2 加载模型

In [2]:
from dual_model import DualModel

# 需要完成前置模型训练
dual_model = DualModel.from_pretrained(
    "../12-sentence_similarity/dual_model/checkpoint-500/"
)
dual_model = dual_model.cuda()
dual_model.eval()
print("匹配模型加载成功！")

/home/pika/App/miniconda3/envs/transformers/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


匹配模型加载成功！


In [3]:
from transformers import AutoTokenizer

tokenzier = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
tokenzier

BertTokenizerFast(name_or_path='hfl/chinese-macbert-base', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

## Step3 将问题编码为向量

In [4]:
import torch
from tqdm import tqdm

questions = data["title"].to_list()
vectors = []
with torch.inference_mode():
    for i in tqdm(range(0, len(questions), 32)):
        batch_sens = questions[i : i + 32]
        inputs = tokenzier(
            batch_sens,
            return_tensors="pt",
            padding=True,
            max_length=128,
            truncation=True,
        )
        inputs = {k: v.to(dual_model.device) for k, v in inputs.items()}
        vector = dual_model.bert(**inputs)[1]  # pooled output
        vectors.append(vector)
vectors = torch.concat(vectors, dim=0).cpu().numpy()
vectors.shape

100%|██████████| 570/570 [00:14<00:00, 39.14it/s]


(18213, 768)

## Step4 创建索引

In [5]:
import faiss

index = faiss.IndexFlatIP(768)
faiss.normalize_L2(vectors)
index.add(vectors)
index

<faiss.swigfaiss_avx2.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x719120cfbd80> >

## Step5 对问题进行向量编码

In [6]:
quesiton = "寻衅滋事"
with torch.inference_mode():
    inputs = tokenzier(
        quesiton, return_tensors="pt", padding=True, max_length=128, truncation=True
    )
    inputs = {k: v.to(dual_model.device) for k, v in inputs.items()}
    vector = dual_model.bert(**inputs)[1]
    q_vector = vector.cpu().numpy()
q_vector.shape

(1, 768)

## Step6 向量匹配(召回)

In [7]:
faiss.normalize_L2(q_vector)
scores, indexes = index.search(q_vector, 10)
print(indexes)
topk_result = data.values[indexes[0].tolist()]  # 从原始数据取出topk的结果
topk_result[:, 0]

[[  539  7292 17072  7828  6441  4377  1739 10990  9819  8229]]


array(['涉嫌寻衅滋事', '两个轻微伤够寻衅滋事', '聚群斗殴', '打架斗殴处罚条例', '敲诈勒索罪', '打架斗殴',
       '民间放高利贷犯法', '容留卖淫罪定义成投资人', '交通事故肇事逃逸', '非法集资大案研究：中晋资本为什么会被定集资诈骗罪'],
      dtype=object)

## Step7 加载交互模型

In [8]:
from transformers import BertForSequenceClassification

# 需要完成前置模型训练
corss_model = BertForSequenceClassification.from_pretrained(
    "../12-sentence_similarity/cross_model/checkpoint-500/"
)
corss_model = corss_model.cuda()
corss_model.eval()
print("模型加载成功！")

模型加载成功！


## Step8 最终预测(排序)

In [9]:
canidate = topk_result[:, 0].tolist()  # question
ques = [quesiton] * len(canidate)
inputs = tokenzier(
    ques, canidate, return_tensors="pt", padding=True, max_length=128, truncation=True
)
inputs = {k: v.to(corss_model.device) for k, v in inputs.items()}
with torch.inference_mode():
    logits = corss_model(**inputs).logits.squeeze()
    result = torch.argmax(logits, dim=-1)
result

tensor(0, device='cuda:0')

In [10]:
canidate_answer = topk_result[:, 1].tolist()  # answer
match_quesiton = canidate[result.item()]
final_answer = canidate_answer[result.item()]
match_quesiton, final_answer

('涉嫌寻衅滋事',
 '说明具有寻衅滋事行为，应受到相应的处罚，行为人情形严重或行为恶劣的涉嫌了寻衅滋事罪。寻衅滋事是指行为人结伙斗殴的、追逐、拦截他人的、强拿硬要或者任意损毁、占用公私财物的、其他寻衅滋事的行为。寻衅滋事罪，是指在公共场所无事生非、起哄闹事，造成公共场所秩序严重混乱的，追逐、拦截、辱骂、恐吓他人，强拿硬要或者任意损毁、占用公私财物，破坏社会秩序，情节严重的行为。对于寻衅滋事行为的处罚：1、《中华人*共和国治安管理处罚法》第二十六条规定，有下列行为之一的，处五日以上十日以下拘留，可以并处五百元以下罚款;情节较重的，处十日以上十五日以下拘留，可以并处一千元以下罚款:(一)结伙斗殴的;(二)追逐、拦截他人的;(三)强拿硬要或者任意损毁、占用公私财物的;(四)其他寻衅滋事行为;2、《中华人*共和国刑法》第二百九十三条有下列寻衅滋事行为之一，破坏社会秩序的，处五年以下有期徒刑、拘役或者管制:(一)随意殴打他人，情节恶劣的;(二)追逐、拦截、辱骂、恐吓他人，情节恶劣的;(三)强拿硬要或者任意损毁、占用公私财物，情节严重的;(四)在公共场所起哄闹事。造成公共场所秩序严重混乱的。纠集他人多次实施前款行为，严重破坏社会秩序的，处五年以上十年以下有期徒刑，可以并处罚金。3、最*人*法*和最*人*检**《关于办理寻衅滋事案件的司法解释》为依法惩治寻衅滋事犯罪，维护社会秩序，最*人*法*会*最*人*检**根据《中华人*共和国刑法》的有关规定，就办理寻衅滋事刑事案件适用法律的若干问题司法解释如下:第一条行为人为寻求刺激、发泄情绪、逞强耍横等，无事生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"。行为人因日常生活中的偶发矛盾纠纷，借故生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"，但矛盾系由被害人故意引发或者被害人对矛盾激化负有主要责任的除外。行为人因婚恋、家庭、邻里、债务等纠纷，实施殴打、辱骂、恐吓他人或者损毁、占用他人财物等行为的，一般不认定为"寻衅滋事"，但经有关部门批评制止或者处理处罚后，继续实施前列行为，破坏社会秩序的除外。第二条随意殴打他人，破坏社会秩序，具有下列情形之一的，应当认定为刑法第二百九十三条第一款第一项规定的"情节恶劣":1、致一人以上轻伤或者二人以上轻微伤的;2、引起他人精神失常、自杀等严重后果的;3、多次随意殴打

In [11]:
topk_result

array([['涉嫌寻衅滋事',
        '说明具有寻衅滋事行为，应受到相应的处罚，行为人情形严重或行为恶劣的涉嫌了寻衅滋事罪。寻衅滋事是指行为人结伙斗殴的、追逐、拦截他人的、强拿硬要或者任意损毁、占用公私财物的、其他寻衅滋事的行为。寻衅滋事罪，是指在公共场所无事生非、起哄闹事，造成公共场所秩序严重混乱的，追逐、拦截、辱骂、恐吓他人，强拿硬要或者任意损毁、占用公私财物，破坏社会秩序，情节严重的行为。对于寻衅滋事行为的处罚：1、《中华人*共和国治安管理处罚法》第二十六条规定，有下列行为之一的，处五日以上十日以下拘留，可以并处五百元以下罚款;情节较重的，处十日以上十五日以下拘留，可以并处一千元以下罚款:(一)结伙斗殴的;(二)追逐、拦截他人的;(三)强拿硬要或者任意损毁、占用公私财物的;(四)其他寻衅滋事行为;2、《中华人*共和国刑法》第二百九十三条有下列寻衅滋事行为之一，破坏社会秩序的，处五年以下有期徒刑、拘役或者管制:(一)随意殴打他人，情节恶劣的;(二)追逐、拦截、辱骂、恐吓他人，情节恶劣的;(三)强拿硬要或者任意损毁、占用公私财物，情节严重的;(四)在公共场所起哄闹事。造成公共场所秩序严重混乱的。纠集他人多次实施前款行为，严重破坏社会秩序的，处五年以上十年以下有期徒刑，可以并处罚金。3、最*人*法*和最*人*检**《关于办理寻衅滋事案件的司法解释》为依法惩治寻衅滋事犯罪，维护社会秩序，最*人*法*会*最*人*检**根据《中华人*共和国刑法》的有关规定，就办理寻衅滋事刑事案件适用法律的若干问题司法解释如下:第一条行为人为寻求刺激、发泄情绪、逞强耍横等，无事生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"。行为人因日常生活中的偶发矛盾纠纷，借故生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"，但矛盾系由被害人故意引发或者被害人对矛盾激化负有主要责任的除外。行为人因婚恋、家庭、邻里、债务等纠纷，实施殴打、辱骂、恐吓他人或者损毁、占用他人财物等行为的，一般不认定为"寻衅滋事"，但经有关部门批评制止或者处理处罚后，继续实施前列行为，破坏社会秩序的除外。第二条随意殴打他人，破坏社会秩序，具有下列情形之一的，应当认定为刑法第二百九十三条第一款第一项规定的"情节恶劣":1、致一人以上轻伤或者二人以上轻微伤的;2、引起他人精神失常、自杀等